# Análise da sequência e das features presentes no NCBI

Esta etapa tem como objetivo aceder ao NCBI e guardar os ficheiros correspondentes aos genes escolhidos, e ainda possivelmente explorar variantes.

Para tal, inicialmente fez-se o import do módulo Entrez, que providencia código para aceder ao NCBI através da World Wide Web.(2)

Para além disso, importou-se o objeto SeqIO, o qual fornece um conjunto de interfaces para trabalhar com  vários formatos de sequências biológicas como ficheiros 'fasta' ou 'genBank'.

In [95]:
from Bio import Entrez
from Bio import SeqIO


Posto isto, efetua-se uma procura no NCBI através do Entrez.esearch pelos nucleotidos relacionados com os genes que se procuram, INS e IGF2.

In [96]:
Entrez.email = "pg50253@alunos.uminho.pt"
handle=Entrez.esearch(db="nucleotide", term="Homo sapiens[Orgn] AND INS[Gene]",idtype="acc")
handle2=Entrez.esearch(db="nucleotide", term="Homo sapiens[Orgn] AND IGF2[Gene]",idtype="acc")
record=Entrez.read(handle)
record2=Entrez.read(handle2)
print("ID List INS:\n",record["IdList"])
print("ID List IGF2:\n",record2["IdList"])
handle.close() 
handle2.close() 

ID List INS:
 ['NG_050578.1', 'NG_007114.1', 'NM_001185098.2', 'NM_001291897.2', 'NM_000207.3', 'NM_001185097.2', 'OM489474.1', 'NC_060935.1', 'NC_000011.10', 'MT335692.1', 'MT335691.1', 'MT335690.1', 'MT335689.1', 'MT335688.1', 'MT335687.1', 'MT501209.1', 'JF909299.1', 'DQ896283.2', 'AH002844.2', 'AH012037.2']
ID List IGF2:
 ['NG_050578.1', 'NG_008849.1', 'NM_001007139.6', 'NM_001291862.3', 'NM_001127598.3', 'NM_001291861.3', 'NM_000612.6', 'NC_060935.1', 'NC_000011.10', 'AH005228.2', 'AH002704.2', 'HM481220.1', 'HM481219.1', 'DQ895970.2', 'AH012037.2', 'CM000262.1', 'CH471158.1', 'BC000531.1', 'NT_086367.4', 'AY399429.1']


Analisando, no próprio NCBI, cada um destes IDs, conclui-se que aquele referente à insulina no Homo sapiens é o 'NG_007114.1' (segundo na lista de IDs); aquele que corresponde ao segundo gene a analisar, IGF2, é o 'NG_008849.1', e ainda, em ambas as listas identifica-se um gene comum (o primeiro de cada lista de IDs), 'NG_050578.1', correspondente a um gene que inclui 2 variantes de transcrição alternada, que alinham com o gene INS na região 5' e com o gene IGF2 na região 3'.

Assim, procede-se à análise das 3 sequências e das suas *features*, recorrendo ao SeqIO.parse().

Analisando primeiro as sequências, pode-se automatizar a recolha de informações como o seu ID, descrição, comprimento da sequência, organismo de origem, e outras anotações (dicionário de informações adicionais acerca da sequência, que permite a adição de mais informação 'não estruturada' à sequência. (3)

In [97]:
Entrez.email = "pg50253@alunos.uminho.pt"
handle = Entrez.efetch(db="nucleotide", rettype="gb", retmode="text", id="NG_007114.1,NG_008849.1,NG_050578.1") 
for seq_record in SeqIO.parse(handle, "genbank"):
    print ("ID: ",seq_record.id, " with ", len(seq_record.features), " features ")
    print (seq_record.description[:100], "...")
    print ("Sequence length: ", len(seq_record))
    print ("from: ", seq_record.annotations["source"],"\n")
    print ("Other annotations:")
    print (" -Molecule type: ", seq_record.annotations["molecule_type"])
    print (" -Topology: ", seq_record.annotations["topology"])
    print (" -Date: ", seq_record.annotations["date"])
    print (" -Sequence version:", seq_record.annotations["sequence_version"])
    print (" -Organism: ", seq_record.annotations["organism"])
    print (" -Taxonomy: ", seq_record.annotations["taxonomy"])
    print ("---------------------------------------" )
handle.close()

ID:  NG_007114.1  with  31  features 
Homo sapiens insulin (INS), RefSeqGene on chromosome 11 ...
Sequence length:  8416
from:  Homo sapiens (human) 

Other annotations:
 -Molecule type:  DNA
 -Topology:  linear
 -Date:  02-JAN-2023
 -Sequence version: 1
 -Organism:  Homo sapiens
 -Taxonomy:  ['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Mammalia', 'Eutheria', 'Euarchontoglires', 'Primates', 'Haplorrhini', 'Catarrhini', 'Hominidae', 'Homo']
---------------------------------------
ID:  NG_008849.1  with  105  features 
Homo sapiens insulin like growth factor 2 (IGF2), RefSeqGene (LRG_1031) on chromosome 11 ...
Sequence length:  27487
from:  Homo sapiens (human) 

Other annotations:
 -Molecule type:  DNA
 -Topology:  linear
 -Date:  13-JAN-2023
 -Sequence version: 1
 -Organism:  Homo sapiens
 -Taxonomy:  ['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Mammalia', 'Eutheria', 'Euarchontoglires', 'Primates', 'Haplorrhini', 'C

De seguida, analisando as *features* da sequência, através dos objetos SeqFeature. Estes permitem guardar informações sobre anotações das sequências de forma estruturada, com uma estrutura baseada no formato da tabela de *features* do GenBank.
Os seus principais atributos são:
   
   • location: localização da feature na sequência (pode ser uma posição, um 
    intervalo, etc.)
   
   • type: diz o tipo da feature (string)
   
   • qualifiers: informação adicional (dicionário)

Assim sendo, realiza-se primeiro o import do objeto SeqFeature

In [98]:
from Bio.SeqFeature import SeqFeature
handle = Entrez.efetch(db="nucleotide", rettype="gb", retmode="text", id="NG_007114.1,NG_008849.1,NG_050578.1") 
for seq_record in SeqIO.parse(handle, "genbank"):
    features=seq_record.features
    tipos_features=[]
    posicoes_features=[]
    qualifiers_features=[]
    num=0
    for feature in features:
        posicoes_features.append(feature.location)
        qualifiers_features.append(feature.qualifiers)
        if feature.type not in tipos_features:
            tipos_features.append(feature.type)
    print("O número de features é: ",len(seq_record.features),"\n")
    print("Os tipos de features são: ",tipos_features,"\n")
    for tipo in tipos_features:
        for feature in features:
            if feature.type==tipo:
                num+=1
        print("O número das features do tipo",tipo,"são:", num)
    print("\n As posições das features são: ",seq_record.features,"\n")
    print("Os qualifiers das features são: ",qualifiers_features,"\n")
    print("-------------------------------------------")
    featcds = [ ]
    for feature in seq_record.features:
        if feature.type == "CDS":
            featcds.append(feature)
    print("Número de features do tipo cds:",len(featcds),"\n")
    print ("Localização das features do tipo cds:")
    for k in featcds:
        print(k.location)
    print("\nIdentificação das proteínas e das suas respetivas sequências: ")
    for k in range(len(featcds)):
        product=featcds[k].qualifiers["product"] 
        print(product)
        translation=featcds[k].qualifiers["translation"]
        print(translation)
        print("")
    print("-------------------------------------------")
    print("-------------------------------------------")
handle.close() 

O número de features é:  31 

Os tipos de features são:  ['source', 'gene', 'mRNA', 'CDS', 'misc_feature', 'misc_RNA', 'exon', 'sig_peptide', 'proprotein', 'mat_peptide'] 

O número das features do tipo source são: 1
O número das features do tipo gene são: 4
O número das features do tipo mRNA são: 9
O número das features do tipo CDS são: 14
O número das features do tipo misc_feature são: 21
O número das features do tipo misc_RNA são: 22
O número das features do tipo exon são: 25
O número das features do tipo sig_peptide são: 27
O número das features do tipo proprotein são: 28
O número das features do tipo mat_peptide são: 31

 As posições das features são:  [SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(8416), strand=1), type='source'), SeqFeature(FeatureLocation(BeforePosition(0), ExactPosition(2266), strand=1), type='gene'), SeqFeature(CompoundLocation([FeatureLocation(BeforePosition(136), ExactPosition(193), strand=1), FeatureLocation(ExactPosition(431), ExactPosition(5

Detetou-se assim, na alínea anterior, o número de features para cada uma das 3 sequências, os tipos de features presentes e o número de features de cada tipo, as posições das features e os qualifiers das features. 

Deu-se ainda destaque às features do tipo cds (codifying sequences), por forma a demonstrar mais funcionalidades e opções de seleção destas ferramentas de BioPython. Com a identificação das features codificantes, foi possível identificar também a proteína codificada, e a sua sequência nucleotídica.

Com a análise realizada nos pontos anteriores, conclui-se que os três genes, "INS", "IGF2" e "INS-IGF2 readthrough" pertencem à espécie Homo sapiens. A sua taxonomia completa começa em “Eucariota”, e termina em “Homo”. São moléculas de DNA todas localizadas no cromossoma 11.

Cada gene codifica um conjunto diferente de proteínas, como a Tirosina 3-monooxigenase, que cataliza a conversão de L-tirosina em L-dihidroxifenilalanina (L-Dopa), que influencia a biosíntese de catecolaminas, dopamina, noradrenaline e adrenalina (4); a preproteina de insulina, que é mais tarde convertida em insulina (5); precursor de insulina,isomorfo 2 que aumenta o efeito do IGF2 durante a embriogénese e desenvolvimento fetal(6); preproteina de IGF2, que é mais tarde convertida em IGF2. De notar que o terceiro gene identificado codifica exatamente o conjunto de proteínas que os outros 2 genes têm a capacidade de codificar, por se tratar de um gene que alternadamente se desdobra em codificante de INS e IGF2.